# Подсчет частотности пар товаров в продуктовых чеках

В файле содержится информация о покупках людей.

* id – означает покупку (в одну покупку входят все товары, купленные пользователем во время 1 похода в магазин)
* Товар – наименование товара
* Количество – число единиц купленного товара

Воспользуйтесь этими данными и выясните, какие пары товаров пользователи чаще всего покупают вместе. По сути, вам необходимо найти паттерны покупок, что позволит оптимизировать размещение продуктов в магазине, для удобства пользователей и увеличения выручки.

In [ ]:
%%capture
# Install postgresql server
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql
!sudo service postgresql start

# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"

# Setup a database with name `tfio_demo` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS db;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE db;'

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
user = 'postgres'
password = 'postgres'
host = 'localhost'
post = 5432
name = 'db'

In [ ]:
point = 'postgresql://{}:{}@{}:{}/{}'.format(user, password, host, post, name)

In [ ]:
con = create_engine(point)

In [ ]:
df_test = pd.read_csv('/content/data_two_columns.csv')

In [ ]:
df_test.shape

(541909, 2)

In [ ]:
df_test.to_sql("sales_test", con, if_exists='replace', index=False, method='multi')

In [ ]:
def select_postgresql(sql):
    return pd.read_sql(sql, con)

In [ ]:
sql = """select s.* from sales_test as s limit 10"""

In [ ]:
print(select_postgresql(sql))

  invoiceno stockcode
0    536365    85123A
1    536365     71053
2    536365    84406B
3    536365    84029G
4    536365    84029E
5    536365     22752
6    536365     21730
7    536366     22633
8    536366     22632
9    536367     84879


In [ ]:
sql = """with tbl_no_duplicates as (select s.invoiceno, 
                                          s.stockcode 
                                    from sales_test as s 
                                    group by s.invoiceno, 
                                            s.stockcode),
      
             tbl_list_code_combinations as (select t.invoiceno, (select array_agg(concat(cast(t1.* as text),', ',cast(t2.* as text)))
                                      from unnest(array_agg(t.stockcode)) as t1 
                                                        cross join unnest(array_agg(t.stockcode)) as t2
                                      where t1.* < t2.*) as agg
                                        from tbl_no_duplicates as t
                                        group by t.invoiceno)

select unnest(t.agg) as couple_stockcode, count(*)
from tbl_list_code_combinations as t
 group by  unnest(t.agg) 
 order by count(*) desc
 limit 10"""

In [ ]:
%%time
print(select_postgresql(sql))

  couple_stockcode  count
0    22386, 85099B    833
1     22697, 22699    784
2    21931, 85099B    733
3    22411, 85099B    683
4     20725, 22383    663
5     20725, 20727    648
6     22726, 22727    646
7     22697, 22698    644
8     22698, 22699    614
9     20725, 22384    613
CPU times: user 289 ms, sys: 35.4 ms, total: 324 ms
Wall time: 51.6 s
